# Create radial averages for pump-probe heatmap

In [ ]:
%pylab qt

In [ ]:
import numpy as np  
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import os
import src.pyabel_polar
import scipy

#### Utility functions

In [ ]:
def generate_hdf_file_name(run_number):
    try:
        hdf_file_path = '/gpfs/bl1/current/processed/timepix_hdf/'
        file_start = "run_"+str(run_number).zfill(4)
        not_file_end = 'rawOnly.hdf5'
        hdf_file = [i for i in os.listdir(hdf_file_path) if os.path.isfile(os.path.join(hdf_file_path,i)) and i.startswith(file_start) and not i.endswith(not_file_end)][0]
        hdf_file_complete_path = hdf_file_path+hdf_file
        assert os.path.isfile(hdf_file_complete_path), 'File does not exist!'
        return hdf_file_complete_path
    except IndexError:
        print("Run",run_number,"do not exist!")

def number_of_trains_from_hdf(hdf_file_complete_path):
    '''Retrun number of recorded FEL trains in HDF file'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        trains = len(h_file['tpx3Times/triggerNr'][:])
    return trains

def data_from_hdf(hdf_file_complete_path, event_type = 'raw'):
    '''Read data from TimePix HDF files
    Choose raw or centroided data, default is centroided data'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        tof = h_file[str(event_type)+'/tof'][:]
        x_pos = h_file[str(event_type)+'/x'][:]
        y_pos = h_file[str(event_type)+'/y'][:]
    number_of_trains_from_hdf(hdf_file_complete_path)
    return tof, x_pos, y_pos

def data_sliced_by_tof(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, event_type = 'raw'):
    '''Slice data with respect to time-of-flight dimension'''
    tof, x_pos, y_pos = data_from_hdf(hdf_file_complete_path, event_type)
    sliced_x_pos = x_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_y_pos = y_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_tof = tof[np.logical_and(tof > tof_start, tof < tof_end)]
    return sliced_tof, sliced_x_pos, sliced_y_pos

def reduce_raw_data(tof, x_pos, y_pos, number_of_events):
    '''Reduce data for visualization'''
    return tof[:number_of_events], x_pos[:number_of_events],y_pos[:number_of_events]

def tof_conversion(tof, time_unit):
    '''Convert time axis'''
    if time_unit == None:
        return tof, 's'
    if time_unit == 'milli':
        return tof*10**3, 'ms'
    if time_unit == 'micro':
        return tof*10**6, 'us'

def plot_tof(tof, tof_start, tof_end, hist_bins = 100, time_unit = None):     
    '''Plot time-of-flight spectrum via histogram'''
    fig = plt.subplots(num = 1)
    plt.clf()
    tof, time_tof_unit = tof_conversion(tof, time_unit)
    tof_start,time_tof_unit= tof_conversion(tof_start, time_unit)
    tof_end, time_tof_unit= tof_conversion(tof_end, time_unit)
    hist_y, hist_x = np.histogram(tof, range = (tof_start, tof_end), bins = hist_bins)
    hist_y = np.append(hist_y, 0)
    plt.plot(hist_x, hist_y)
    plt.title('histogram: time-of-flight')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('number of events')
    plt.show()
    return hist_y, hist_x
    
def vmi_image(x_pos, y_pos, show_image = True):
    '''Display VMI image - cmax empirically found to surpress hot pixel '''
    fig = plt.figure(num = 6)
    plt.clf()
    counts, xbins, ybins, image = plt.hist2d(x_pos, y_pos,bins=np.linspace(0, 256, 257)) #, cmax= 1000)
    plt.colorbar()
    plt.xlabel('x_pos [px]')
    plt.ylabel('y_pos [px]')
    if show_image == False:
        plt.close()
    return counts
    
def display_tof_and_vmi_of_tof_interval(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, hist_bins = 100, time_unit = None, event_type = 'raw'):
    '''Display VMI Image and time-of-flight spetrum'''
    tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start, tof_end, event_type)
    plot_tof(tof, tof_start, tof_end, hist_bins, time_unit)
    vmi_image(x_pos,y_pos)

    
def transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius, non_plot = False):
    
    counts = vmi_image(x_pos, y_pos, show_image = False)
    image_cart = np.flipud(counts.transpose())
    
    image_polar, r_grid, theta_grid = src.pyabel_polar.reproject_image_into_polar(image_cart, origin=(x_center,y_center))
    radial_ave = np.sum(image_polar, axis=1)
    
    if non_plot == False:
        fig = plt.figure(figsize = (10,10),num = 6)
        plt.clf()
        plt.imshow(image_cart)
        plt.scatter(x_center, y_center, color='r')
        plt.gcf().gca().add_artist(plt.Circle((x_center, y_center), radius, color='r', fill=False))
        plt.xlabel('x_posr [px]')
        plt.ylabel('y_posr [px]')
        plt.title('VMI image')

        fig = plt.figure(num = 7)
        plt.clf()
        plt.plot(radial_ave,'r-')
        plt.title('Radial average')
        plt.xlabel('r [px]')
        plt.ylabel('counts')
        plt.show()
    
    return radial_ave

def get_delay_stage_pos_from_txt_file(run_interval):
    runs = []
    delay_pos = []
    filepath = '/home/bl1user/Desktop/erk20919/CAMP/beamtime/run_pp-delay.txt'
    with open(filepath) as fp:
        for cnt, line in enumerate(fp):
            tt = [x.strip() for x in line.split(',')]
            runs.append(int(tt[0]))
            delay_pos.append(float(tt[1]))
    delay_pos = delay_pos[runs.index(run_interval[0]):runs.index(run_interval[1])+1]
    runs = runs[runs.index(run_interval[0]):runs.index(run_interval[1])+1]
    return runs, delay_pos

def save_all_delay_for_fragment(file_prefix, fragment ,run_interval, tof_start,tof_end, x_center, y_center, radius, timepix_event_type):
    runs_file , delay_stage = get_delay_stage_pos_from_txt_file(run_interval)
    for i in range(run_interval[0], run_interval[1]+1):
        print('Working on ...{}/{}'.format((i-run_interval[0])+1,(run_interval[1]+1-run_interval[0])))
        hdf_file = generate_hdf_file_name(i)
        tof, x_pos, y_pos = data_sliced_by_tof(hdf_file, tof_start , tof_end, event_type = timepix_event_type)
        radial_average = transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius, non_plot = True) 
        number_of_trains = number_of_trains_from_hdf(hdf_file)
        
        save_to_path = 'intermediate_data/'
        save_file_name = str(file_prefix)+"_"+str(fragment)+"_"+str(delay_stage[i-run_interval[0]])+"_Run"+str(i)
        np.savez(save_to_path+save_file_name, radial_average=radial_average, fragment= fragment, delay_stage = delay_stage[i-run_interval[0]], number_of_trains = number_of_trains, run = i)


### Define source file & describe measurment

In [ ]:
run_number = 364
timepix_event_type = 'raw'

hdf_file = generate_hdf_file_name(run_number)

tof_start = 1.5E-6
#tof_end = 12E-6
tof_end = 10E-6

### Complete time-of-flight

In [ ]:
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file, tof_start , tof_end, event_type = timepix_event_type)
plot_tof(tof, tof_start, tof_end,  hist_bins = 1000, time_unit = 'micro');

### Slice time-of-flight dimension and create VMI image

In [ ]:
# I2+ 
# tof_start, tof_end = 7.60E-6, 8.01E-6

# I3+ 
# tof_start, tof_end = 6.4E-6, 6.7E-6

# I4+ 
# tof_start, tof_end = 5.75E-6, 5.95E-6

# I5+ 
# tof_start, tof_end = 5.2E-6, 5.4E-6

# I6+ 
#tof_start, tof_end = 4.87E-6, 4.98E-6

# I6+ zoomed (half voltages)
#tof_start, tof_end = 6.4E-6, 6.7E-6

# I6+ (double voltages)
tof_start, tof_end = 3.71E-6, 3.85E-6


display_tof_and_vmi_of_tof_interval(hdf_file, tof_start, tof_end, event_type = timepix_event_type, time_unit = 'micro')

### Chose center and calc radial average

In [ ]:
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file, tof_start , tof_end, event_type = timepix_event_type)

In [ ]:
# fragment = 'I2+'
# x_center, y_center = 134, 118        
# radius = 80

# fragment = 'I3+'      
# x_center, y_center = 131, 127          
# radius = 18

# fragment = 'I4+'
# x_center, y_center = 131, 127          
# radius = 18

# fragment = 'I5+'
# x_center, y_center = 131, 127          
# radius = 18

fragment = 'I6+'
x_center, y_center = 135, 256-110   # center with double voltages and Helium jet offset
#x_center, y_center = 135, 256-119   # new center with corrected coordinate system?
#x_center, y_center = 135, 110        
#x_center, y_center = 135, 119
radius = 30

radial_average = transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius)

## Save to intermediate results 

In [ ]:
file_prefix = '800nm_Iodopyridine_I6p' 
#run_interval = [201, 203]
#run_interval = [359, 361] # 362 is missing!
run_interval = [369, 372]

In [ ]:
save_all_delay_for_fragment(file_prefix, fragment ,run_interval, tof_start, tof_end, x_center, y_center, radius, timepix_event_type)

In [ ]:
tof_start = 0E-6
tof_end = 14E-6

run_number1 = 201
timepix_event_type = 'raw'
hdf_file = generate_hdf_file_name(run_number1)
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file, tof_start , tof_end, event_type = timepix_event_type)
tt1, tt11 = plot_tof(tof, tof_start, tof_end,  hist_bins = 500, time_unit = 'micro')
tt1 = tt1 / number_of_trains_from_hdf(hdf_file)
tt1 = np.append(tt1, 0)

run_number2 = 252
timepix_event_type = 'raw'
hdf_file = generate_hdf_file_name(run_number2)
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file, tof_start , tof_end, event_type = timepix_event_type)
tt2, tt22 = plot_tof(tof, tof_start, tof_end,  hist_bins = 500, time_unit = 'micro')
tt2 = tt2 / number_of_trains_from_hdf(hdf_file)
tt2 = np.append(tt2, 0)

run_number3 = 281
timepix_event_type = 'raw'
hdf_file = generate_hdf_file_name(run_number3)
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file, tof_start , tof_end, event_type = timepix_event_type)
tt3, tt33 = plot_tof(tof, tof_start, tof_end,  hist_bins = 500, time_unit = 'micro')
tt3 = tt3 / number_of_trains_from_hdf(hdf_file)
tt3 = np.append(tt3, 0)

run_number4 = 283
timepix_event_type = 'raw'
hdf_file = generate_hdf_file_name(run_number4)
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file, tof_start , tof_end, event_type = timepix_event_type)
tt4, tt44 = plot_tof(tof, tof_start, tof_end,  hist_bins = 500, time_unit = 'micro')
tt4 = tt4 / number_of_trains_from_hdf(hdf_file)
tt4 = np.append(tt4, 0)

plt.clf()
plt.plot(tt11, tt1,'b', label = run_number1)
plt.plot(tt22, tt2,'r', label = run_number2)
plt.plot(tt33, tt3,'k', label = run_number3)
plt.plot(tt44, tt4,'k', label = run_number4)
plt.xlabel('ToF[mu s]')
plt.legend()

In [ ]:
plt.clf()
plt.plot(tt11, tt1,'b', label = run_number1)
plt.plot(tt22, tt2,'r', label = run_number2)
plt.plot(tt33, tt3,'k', label = run_number3)
# plt.plot(tt44, tt4,'g', label = run_number4)
plt.xlabel('ToF[mu s]')
plt.legend()

In [ ]:
### 2nd fragment

### Slice time-of-flight dimension and create VMI image

In [ ]:
# N+ Iodopyridine (double voltages)
#tof_start, tof_end = 3.35E-6, 3.50E-6

# I2+ Iodopyridine (double voltages)
tof_start, tof_end = 5.67E-6, 6.010E-6


display_tof_and_vmi_of_tof_interval(hdf_file, tof_start, tof_end, event_type = timepix_event_type, time_unit = 'micro')

### Chose center and calc radial average

In [ ]:
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file, tof_start , tof_end, event_type = timepix_event_type)

In [ ]:
fragment = 'I2+'
x_center, y_center = 135, 256-98   # N+ center with double voltages and Helium jet offset
#x_center, y_center = 135, 98        
radius = 30

#fragment = 'N+'
#x_center, y_center = 135, 256-115   # N+ center with double voltages and Helium jet offset
#x_center, y_center = 135, 115        
#radius = 30

radial_average = transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius)

## Save to intermediate results 

In [ ]:
file_prefix = '800nm_Iodopyridine_I2p' 
#run_interval = [359, 361] # 362 is missing!
run_interval = [363, 372]

In [ ]:
save_all_delay_for_fragment(file_prefix, fragment ,run_interval, tof_start, tof_end, x_center, y_center, radius, timepix_event_type)